First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [2]:
model.load_state_dict(torch.load('road_following_model.pth'))

<All keys matched successfully>

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [4]:
torch.save(model_trt.state_dict(), 'road_following_model_trt.pth')

Load the optimized model by executing the cell below

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))

<All keys matched successfully>

Create the racecar class

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [3]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = 1.3
STEERING_BIAS = 0.4

car.throttle = 0.2
car.throttle_gain = 0.2
car.steering = 0
car.steering_gain = -0.5
car.steering_offset = 0

# back wheel setting
# car.throttle means the maximum speed of back wheel
# car.throttle_gain means the maximum limitation speed of back wheel

# front wheel setting
# car.steering_gain means the maximum rotation of front wheel
# car.steering_offset means the init shifting of front wheel

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    pid_steering = x * STEERING_GAIN + STEERING_BIAS
    print("[OUTPUT] AI-Output:{} PID-Steering:{}".format(x,pid_steering))
    car.steering = pid_steering

[OUTPUT] AI-Output:-0.2401123046875 PID-Steering:0.08785400390625003
[OUTPUT] AI-Output:-0.23583984375 PID-Steering:0.09340820312500003
[OUTPUT] AI-Output:-0.240234375 PID-Steering:0.08769531250000001
[OUTPUT] AI-Output:-0.237060546875 PID-Steering:0.09182128906250003
[OUTPUT] AI-Output:-0.2386474609375 PID-Steering:0.08975830078125002
[OUTPUT] AI-Output:-0.239990234375 PID-Steering:0.0880126953125
[OUTPUT] AI-Output:-0.2430419921875 PID-Steering:0.08404541015625
[OUTPUT] AI-Output:-0.23974609375 PID-Steering:0.08833007812499999
[OUTPUT] AI-Output:-0.2381591796875 PID-Steering:0.09039306640625
[OUTPUT] AI-Output:-0.238525390625 PID-Steering:0.08991699218749999
[OUTPUT] AI-Output:-0.239013671875 PID-Steering:0.08928222656250001
[OUTPUT] AI-Output:-0.2386474609375 PID-Steering:0.08975830078125002
[OUTPUT] AI-Output:-0.23388671875 PID-Steering:0.095947265625
[OUTPUT] AI-Output:-0.2373046875 PID-Steering:0.09150390624999999
[OUTPUT] AI-Output:-0.2386474609375 PID-Steering:0.089758300781250